In [1]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm_notebook as tqdm
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import SnowballStemmer
from sklearn.metrics import classification_report
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ivky1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
stop_words = set(stopwords.words("english"))

In [3]:
def preprocessing(row,regex,stemming=False):
    text = re.sub(regex, ' ', row)
    text = text.lower()
    text = text.split()
    if stemming:
        snowball_stemmer = SnowballStemmer('english')
        text = [snowball_stemmer.stem(word) for word in text if not word in stop_words]   
    text = " ".join(text)
    return text

In [4]:
training=pd.read_csv(r"C:\Users\ivky1\University\Year_1\Semester_2\Text Mining\Project\TextMining\training_set.txt", sep="\t")
validation=pd.read_csv(r"C:\Users\ivky1\University\Year_1\Semester_2\Text Mining\Project\TextMining\dev_set.txt", sep="\t")

In [5]:
training_processed=training.copy()
validation_processed=validation.copy()
training_processed.sentence=training.apply(lambda x: preprocessing(x.sentence,'[^a-zA-Z0-9!?]'),axis=1)
validation_processed.sentence=validation.apply(lambda x: preprocessing(x.sentence,'[^a-zA-Z0-9!?]'),axis=1)

In [6]:
training_processed_baseline=training.copy()
validation_processed_baseline=validation.copy()
training_processed_baseline.sentence=training.apply(lambda x: preprocessing(x.sentence,'[^a-zA-Z0-9!?]',stemming=True),axis=1)
validation_processed_baseline.sentence=validation.apply(lambda x: preprocessing(x.sentence,'[^a-zA-Z0-9!?]',stemming=True),axis=1)

In [7]:
cv = CountVectorizer()
cv_baseline=CountVectorizer()

In [8]:
X_train_baseline = cv_baseline.fit_transform(training_processed_baseline.sentence)
y_train_baseline = training_processed_baseline.emotion
X_val_baseline = cv_baseline.transform(validation_processed_baseline.sentence)
y_val_baseline = validation_processed_baseline.emotion

In [9]:
X_train = cv.fit_transform(training_processed.sentence)
y_train = training_processed.emotion
X_val = cv.transform(validation_processed.sentence)
y_val = validation_processed.emotion

In [10]:
def evaluate_model(model,xtrain,ytrain,xval,yval):
    model.fit(xtrain,ytrain)
    temp_dict={"train":{},"val":{}}
    for x,y,dataset in [(xtrain,ytrain,"train"),(xval,yval,"val")]:
        temp_dict[dataset]["accuracy"]=accuracy_score(y,model.predict(x))
        temp_dict[dataset]["f1"]=f1_score(y,model.predict(x), average = 'weighted')
        temp_dict[dataset]["precision"]=precision_score(y,model.predict(x), average = 'weighted')
        temp_dict[dataset]["recall"]=recall_score(y,model.predict(x), average = 'weighted')
    print(classification_report(validation['emotion'], model.predict(x)))
    return pd.DataFrame(temp_dict)

In [11]:
#pass in preprocessed data
def evaluate_per_label(model,data,cv):
    temp_dict={"accuracy":{},"f1":{},"precision":{},"recall":{}}
    for i in range(1,9):
        temp=data[data.emotion==i]
        temp_dict["accuracy"][i]=accuracy_score(temp.emotion,model.predict(cv.transform(temp.sentence)))
        temp_dict["f1"][i]=f1_score(temp.emotion,model.predict(cv.transform(temp.sentence)), average = 'weighted')
        temp_dict["precision"][i]=precision_score(temp.emotion,model.predict(cv.transform(temp.sentence)), average = 'weighted')
        temp_dict["recall"][i]=recall_score(temp.emotion,model.predict(cv.transform(temp.sentence)), average = 'weighted')  
    return pd.DataFrame(temp_dict)

In [12]:
mnb=MultinomialNB()
svc_baseline=SVC(kernel="linear")
svc=SVC(kernel="linear",C=0.5)
sgd=SGDClassifier( loss="log")

In [13]:
evaluate_model(svc,X_train,y_train,X_val,y_val)

              precision    recall  f1-score   support

           1       0.37      0.60      0.46       211
           2       0.40      0.48      0.44       170
           3       0.25      0.16      0.19        77
           4       0.43      0.28      0.34       104
           5       0.38      0.40      0.39        97
           6       0.36      0.23      0.28        87
           7       0.47      0.28      0.35        96
           8       0.39      0.30      0.34       158

    accuracy                           0.38      1000
   macro avg       0.38      0.34      0.35      1000
weighted avg       0.38      0.38      0.37      1000



,train,val
accuracy,0.668071,0.383000
f1,0.666308,0.370386
precision,0.686567,0.384995
recall,0.668071,0.383000


In [14]:
#baseline
evaluate_model(svc_baseline,X_train_baseline,y_train_baseline,X_val_baseline,y_val_baseline)

              precision    recall  f1-score   support

           1       0.35      0.60      0.44       211
           2       0.35      0.35      0.35       170
           3       0.19      0.13      0.16        77
           4       0.31      0.23      0.27       104
           5       0.40      0.40      0.40        97
           6       0.28      0.18      0.22        87
           7       0.29      0.18      0.22        96
           8       0.34      0.27      0.30       158

    accuracy                           0.33      1000
   macro avg       0.31      0.29      0.29      1000
weighted avg       0.32      0.33      0.32      1000



,train,val
accuracy,0.645214,0.333000
f1,0.643962,0.317417
precision,0.665066,0.323953
recall,0.645214,0.333000


In [15]:
evaluate_per_label(svc,validation_processed,cv)

c:\Users\ivky1\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ivky1\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ivky1\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ivky1\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1327: Un

,accuracy,f1,precision,recall
1,0.601896,0.751479,1.0,0.601896
2,0.482353,0.650794,1.0,0.482353
3,0.155844,0.269663,1.0,0.155844
4,0.278846,0.436090,1.0,0.278846
5,0.402062,0.573529,1.0,0.402062
6,0.229885,0.373832,1.0,0.229885
7,0.281250,0.439024,1.0,0.281250
8,0.297468,0.458537,1.0,0.297468


In [16]:
svc_baseline.fit(X_train_baseline,y_train_baseline)
evaluate_per_label(svc_baseline,validation_processed_baseline,cv_baseline)

c:\Users\ivky1\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ivky1\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ivky1\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ivky1\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1327: Un

,accuracy,f1,precision,recall
1,0.597156,0.747774,1.0,0.597156
2,0.347059,0.515284,1.0,0.347059
3,0.129870,0.229885,1.0,0.129870
4,0.230769,0.375000,1.0,0.230769
5,0.402062,0.573529,1.0,0.402062
6,0.183908,0.310680,1.0,0.183908
7,0.177083,0.300885,1.0,0.177083
8,0.265823,0.420000,1.0,0.265823


In [17]:
svc.fit(X_train,y_train)
prediction_val=svc.predict(X_val)

In [18]:
validation["prediction"]=prediction_val

In [19]:
validation[["sentence","prediction"]]

,sentence,prediction
0,What happens to the gold in our safe ?,4
1,Natural to get cold feet .,1
2,"Not very lucky , is he ?",5
3,I'm just a little anxious to get up there and ...,1
4,Did you think we don't know about your affair ...,1
...,...,...
995,All I ask of you is be careful .,4
996,"You don't like jazz , pal ?",1
997,Put it on .,1
998,Can you ever imagine [PERSON] being in a spot ...,5
